fair_facts_v2:

The Framework for Assessing Changes To Sea-level (FACTS) is an open-source modular, scalable, and extensive framework for global mean, regional, and extreme sea level projection that is designed to support the characterization of ambiguity in sea-level projections. It is designed so users can easily explore deep uncertainty by investigating the implications on GMSL, RSL, and ESL of different choices for different processes. Its modularity allows components to be represented by either simple or complex model. Because it is built upon the Radical-PILOT computing stack, different modules can be dispatched for execution on resources appropriate to their computational complexity.

FACTS is being developed by the Earth System Science & Policy Lab and the RADICAL Research Group at Rutgers University. FACTS is released under the MIT License.

Project Github Page: https://github.com/radical-collaboration/facts

In [1]:
import sys
# Add a path to the search list
sys.path.insert(0, '/discover/nobackup/projects/sealevel/facts2.0')

In [2]:
import os

# Get and print the current working directory (optional, for verification)
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# Change the current working directory to a new path
new_directory_path = "/discover/nobackup/projects/sealevel/facts2.0" # Example for Linux/macOS
# For Windows, you can use forward slashes or a raw string (see below)

try:
    os.chdir(new_directory_path)
    print(f"Directory successfully changed to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Directory not found: {new_directory_path}")
except Exception as e:
    print(f"An error occurred: {e}")


Current working directory: /gpfsm/dnb06/projects/p151/gtamkin/facts2.0/notebooks
Directory successfully changed to: /gpfsm/dnb06/projects/p153/facts2.0


In [3]:
import asyncio
import logging
import time
import os
import shlex

from radical.asyncflow import WorkflowEngine
from radical.asyncflow import ConcurrentExecutionBackend

from concurrent.futures import ThreadPoolExecutor

from radical.asyncflow.logging import init_default_logger

logger = logging.getLogger(__name__)

In [4]:
async def main():
    init_default_logger(logging.DEBUG)

    # Create backend and workflow
    engine = await ConcurrentExecutionBackend(ThreadPoolExecutor())
    flow = await WorkflowEngine.create(engine)
    
    # Ensure output directories exist
    def setup_directories():
        os.makedirs('./data/output/fair', exist_ok=True)
        os.makedirs('./data/output/lws', exist_ok=True)
        os.makedirs('./data/output/sterodynamics', exist_ok=True)

    @flow.executable_task
    async def fair_task():
        """FAIR temperature model task"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', './data/input:/input',
            '--bind', './data/output/fair:/output',
            './containers/fair-temperature.sif',
            'fair-temperature',
            '--pipeline-id=1234',
            '--output-oceantemp-file=/output/oceantemp.nc',
            '--nsamps=20',
            '--output-ohc-file=/output/ohc.nc',
            '--output-gsat-file=/output/gsat.nc',
            '--output-climate-file=/output/climate.nc',
            '--rcmip-file=/input/rcmip/rcmip-emissions-annual-means-v5-1-0.csv',
            '--param-file=/input/parameters/fair_ar6_climate_params_v4.0.nc'
        ]
        return shlex.join(cmd)

    @flow.executable_task
    async def lws_task():
        """Land Water Storage task - can run independently of FAIR"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', './data/input:/input',
            '--bind', './data/output/lws:/output',
            './containers/ssp-landwaterstorage.sif',
            'ssp-landwaterstorage',
            '--pipeline-id=1234',
            '--nsamps=20',
            '--output-gslr-file=/output/gslr.nc',
            '--output-lslr-file=/output/lslr.nc',
            '--location-file=/input/location.lst',
            '--pophist-file=/input/UNWPP2012 population historical.csv',
            '--reservoir-file=/input/Chao2008 groundwater impoundment.csv',
            '--popscen-file=/input/ssp_iam_baseline_popscenarios2100.csv',
            '--gwd-file=/input/Konikow2011 GWD.csv',
            '--gwd-file=/input/Wada2012 GWD.csv',
            '--gwd-file=/input/Pokhrel2012 GWD.csv',
            '--fp-file=/input/REL_GROUNDWATER_NOMASK.nc'
        ]
        return shlex.join(cmd)

    @flow.executable_task
    async def sterodynamics_task(fair_task):
        """Sterodynamics task - depends on FAIR output"""
        cmd = [
            '/usr/local/other/singularity/4.0.3/bin/singularity', 'exec',
            '--bind', './data/output/fair:/fair',
            '--bind', './data/input:/input',
            '--bind', './data/output/sterodynamics:/output',
            '--nv',
            './containers/tlm-sterodynamics.sif',
            'tlm-sterodynamics',
            '--pipeline-id=1234',
            '--scenario=ssp585',
            '--nsamps=20',
            '--model-dir=/input/cmip6/',
            '--location-file=/input/location.lst',
            '--output-lslr-file=/output/lslr.nc',
            '--output-gslr-file=/output/gslr.nc',
            '--expansion-coefficients-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n18_expcoefs.nc',
            '--gsat-rmses-file=/input/scmpy2LM_RCMIP_CMIP6calpm_n17_gsat_rmse.nc',
            '--climate-data-file=/fair/climate.nc'
        ]
        return shlex.join(cmd)

    async def run_climate_workflow(pipeline_id):
        """Run the complete climate workflow"""
        logger.info(f'Starting climate workflow {pipeline_id} at {time.time()}')

        # Setup directories
        setup_directories()

        # Start FAIR and LWS tasks (they can run in parallel)
        fair_future = fair_task()
        lws_future = lws_task()

        # Wait for FAIR to complete (sterodynamics depends on it)
        fair_result = await fair_future
        logger.info(f'FAIR task completed for pipeline {pipeline_id}')

        # Start sterodynamics task (depends on FAIR output)
        sterodynamics_future = sterodynamics_task(fair_future)

        # Wait for all tasks to complete
        lws_result = await lws_future
        sterodynamics_result = await sterodynamics_future

        logger.info(f'Climate workflow {pipeline_id} finished at {time.time()}')

        return {
            'fair': fair_result,
            'lws': lws_result,
            'sterodynamics': sterodynamics_result
        }

    # Run workflow(s)
    results = await run_climate_workflow(1)
    logger.info("All workflows completed successfully")
    logger.info(results)
    await flow.shutdown()

# Just call it with await in Jupyter
await main()


2026-02-09 10:33:52.399 │ INFO │ [root] │ Logger configured successfully - Console: DEBUG, File: disabled (N/A), Structured: disabled, Style: modern
2026-02-09 10:33:52.400 │ INFO │ [execution.backend(concurrent)] │ ThreadPoolExecutor execution backend started successfully
2026-02-09 10:33:52.401 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGHUP
2026-02-09 10:33:52.401 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGTERM
2026-02-09 10:33:52.401 │ DEBUG │ [workflow_manager] │ Registered signal handler for SIGINT
2026-02-09 10:33:52.402 │ DEBUG │ [workflow_manager] │ Started run component
2026-02-09 10:33:52.402 │ INFO │ [main] │ Starting climate workflow 1 at 1770651232.402521
2026-02-09 10:33:52.406 │ DEBUG │ [workflow_manager] │ Registered task: 'fair_task' with id of task.000001
2026-02-09 10:33:52.406 │ DEBUG │ [workflow_manager] │ Registered task: 'lws_task' with id of task.000002
2026-02-09 10:33:52.407 │ DEBUG │ [workflow_manager] │ Ready to subm